In [1]:
from unsloth import FastLanguageModel
max_seq_length = 1024

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/simpissa/multi-turn-grpo/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.8: Fast Gemma3 patching. Transformers: 4.53.3.
   \\   /|    NVIDIA GeForce RTX 3060 Laptop GPU. Num GPUs = 1. Max memory: 6.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    finetune_vision_layers     = False,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 8,
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    random_state = 42,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [3]:
from EnvWorker import EnvWorker
from datasets import Dataset
import torch
import torch.multiprocessing as mp
from torch.multiprocessing import Queue

In [4]:
# def create_batch(batch_size, group_size, game):

#     from copy import deepcopy
#     import textarena as ta
#     import nltk
#     nltk.download('averaged_perceptron_tagger_eng')

#     envs = []
#     for _ in range(batch_size // group_size):
#         env = ta.wrappers.LLMObservationWrapper(env=ta.make(game))
#         env.reset(num_players=1)
#         for _ in range(group_size):
#             envs.append(deepcopy(env))
#     return envs



In [5]:
# # filter action out of completion
# def extract_action(completion: str):
#     return completion
    



In [6]:
# # given a single scalar reward and number of turns T, return a (1, T) tensor with rewards
# def calculate_rewards(reward: int, turns: int):
#     return reward * torch.ones((1, turns))

In [7]:
from MultiTurnUtils import create_batch, extract_action, calculate_reward

worker = EnvWorker(extract_action, calculate_reward)
worker.create_batch = create_batch

In [8]:
batch_size = 2
group_size = 2
steps = 10
game = "Wordle-v0"

system_prompt = "You are playing Wordle."

In [9]:
ctx = mp.get_context("spawn")

prompt_q = ctx.Queue()
completion_q = ctx.Queue()
training_q = ctx.Queue()

p = ctx.Process(target=worker, args=(game, batch_size, group_size, steps, prompt_q, completion_q, training_q))
p.start()

# mp.spawn(worker, args=(batch_size, group_size, steps, prompt_q, completion_q, training_q), 
#                nprocs=1, join=False, daemon=False, start_method="spawn")

In [10]:
def update_prompt(batch):
    observations = prompt_q.get()
    new_prompts = [
        [{"role": "system", "content": system_prompt},
        {"role": "user",   "content": observation}] for observation in observations
    ]

    return {
        'prompt': new_prompts,
    }

ds = Dataset.from_dict({
    "dummy": [0]
})
ds.set_transform(update_prompt)

Parameter 'transform'=<function update_prompt at 0x7f37b3dd7e20> of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [11]:
max_prompt_length = 256
steps = 10

from trl import GRPOConfig
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1,
    num_generations = 2,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1,
    max_steps = steps,
    save_steps = 10,
    max_grad_norm = 0.1,
    report_to = "none",
    output_dir = "outputs",
)
training_args.per_device_train_batch_size = 1
training_args.num_generations = 1
training_args.steps_per_generation = 1

In [12]:
from MultiTurnGRPOTrainer import MultiTurnGRPOTrainer
trainer = MultiTurnGRPOTrainer(
    group_size = 2,
    model = model,
    processing_class = tokenizer,
    args = training_args,
    train_dataset = ds,
    completion_q=completion_q,
    training_q=training_q,
    reward_funcs = []
)

In [13]:
# trainer.num_generations

In [14]:
from torch.utils.data import SequentialSampler
class InfiniteSampler(SequentialSampler):
    def __init__(self, steps, **kwargs):
        self.steps = steps
        super().__init__(**kwargs)

    def __iter__(self):
        while True:
            # yield from range(len(self.data_source))
            yield 0

    def __len__(self):
        return self.steps

In [15]:
from typing import Optional


def _get_train_sampler(self, dataset: Optional[Dataset] = None):
    if dataset is None:
        dataset = self.train_dataset
    return InfiniteSampler(steps, data_source=dataset)
    # return SequentialSampler(dataset)

MultiTurnGRPOTrainer._get_train_sampler = _get_train_sampler

In [16]:
# def _get_train_sampler(self, dataset = None):
#     return None

# MultiTurnGRPOTrainer._get_train_sampler = _get_train_sampler

In [17]:
from torch.utils.data import DataLoader
def get_train_dataloader(self):
    # if self.train_dataset is None:
    #     raise ValueError("Trainer: training requires a train_dataset.")

    train_dataset = self.train_dataset
    data_collator = self.data_collator
    # if is_datasets_available() and isinstance(train_dataset, datasets.Dataset):
    #     train_dataset = self._remove_unused_columns(train_dataset, description="training")
    # else:
    #     data_collator = self._get_collator_with_removed_columns(data_collator, description="training")

    dataloader_params = {
        "batch_size": self._train_batch_size * self.args.steps_per_generation,  # < this is the change
        "collate_fn": data_collator,
        "num_workers": self.args.dataloader_num_workers,
        "pin_memory": self.args.dataloader_pin_memory,
        "persistent_workers": self.args.dataloader_persistent_workers,
    }

    dataloader_params["sampler"] = self._get_train_sampler()
    dataloader_params["drop_last"] = self.args.dataloader_drop_last
    # if version.parse(transformers.__version__) >= version.parse("4.52.0"):
    #     # from transformers 4.52.0, the `seed_worker` requires the `num_workers` and `rank` arguments
    #     dataloader_params["worker_init_fn"] = partial(
    #         seed_worker, num_workers=self.args.dataloader_num_workers, rank=self.args.process_index
    #     )
    # else:
    #     dataloader_params["worker_init_fn"] = seed_worker
    dataloader_params["prefetch_factor"] = self.args.dataloader_prefetch_factor

    return DataLoader(train_dataset, **dataloader_params)

MultiTurnGRPOTrainer.get_train_dataloader = get_train_dataloader

In [18]:
torch._dynamo.config.cache_size_limit = 32

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 6,522,880 of 1,006,408,832 (0.65% trained)
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/simpissa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /home/simpissa/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/simpissa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /home/simpissa/nltk_data...
[nltk_data]   Package words is already

Observation: ["\n[GAME] You are Playing Wordle.\nA secret 5-letter word has been chosen. You have 6 attempts to guess it.\nFor each guess, wrap your word in square brackets (e.g., '[apple]').\nFeedback for each letter will be given as follows:\n  - G (green): correct letter in the correct position\n  - Y (yellow): letter exists in the word but in the wrong position\n  - X (wrong): letter is not in the word\nEnter your guess to begin.\n", "\n[GAME] You are Playing Wordle.\nA secret 5-letter word has been chosen. You have 6 attempts to guess it.\nFor each guess, wrap your word in square brackets (e.g., '[apple]').\nFeedback for each letter will be given as follows:\n  - G (green): correct letter in the correct position\n  - Y (yellow): letter exists in the word but in the wrong position\n  - X (wrong): letter is not in the word\nEnter your guess to begin.\n"]

Prompt: ["<bos><start_of_turn>user\nYou are playing Wordle.\n\n[GAME] You are Playing Wordle.\nA secret 5-letter word has been ch

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.



Prompt: ["<bos><start_of_turn>user\nYou are playing Wordle.\n\n[GAME] You are Playing Wordle.\nA secret 5-letter word has been chosen. You have 6 attempts to guess it.\nFor each guess, wrap your word in square brackets (e.g., '[apple]').\nFeedback for each letter will be given as follows:\n  - G (green): correct letter in the correct position\n  - Y (yellow): letter exists in the word but in the wrong position\n  - X (wrong): letter is not in the word\nEnter your guess to begin.\n\n[GAME] [dfs]\n[GAME] You attempted an invalid move. Reason: Your word must be exactly 5 letters. Please resubmit a valid move and remember to follow the game rules to avoid penalties.<end_of_turn>\n<start_of_turn>model\n", "<bos><start_of_turn>user\nYou are playing Wordle.\n\n[GAME] You are Playing Wordle.\nA secret 5-letter word has been chosen. You have 6 attempts to guess it.\nFor each guess, wrap your word in square brackets (e.g., '[apple]').\nFeedback for each letter will be given as follows:\n  - G (

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl
1,0.000000,0.000000,0,0,0,0,0,0,0,0,nan
2,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,nan
3,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,nan



Prompt: ["<bos><start_of_turn>user\nYou are playing Wordle.\n\n[GAME] You are Playing Wordle.\nA secret 5-letter word has been chosen. You have 6 attempts to guess it.\nFor each guess, wrap your word in square brackets (e.g., '[apple]').\nFeedback for each letter will be given as follows:\n  - G (green): correct letter in the correct position\n  - Y (yellow): letter exists in the word but in the wrong position\n  - X (wrong): letter is not in the word\nEnter your guess to begin.\n\n[GAME] [dfs]\n[GAME] You attempted an invalid move. Reason: Your word must be exactly 5 letters. Please resubmit a valid move and remember to follow the game rules to avoid penalties.\n[GAME] You are Playing Wordle.\nA secret 5-letter word has been chosen. You have 6 attempts to guess it.\nFor each guess, wrap your word in square brackets (e.g., '[apple]').\nFeedback for each letter will be given as follows:\n  - G (green): correct letter in the correct position\n  - Y (yellow): letter exists in the word bu

Process SpawnProcess-1:
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/simpissa/multi-turn-grpo/EnvWorker.py", line 35, in __call__
    completions = completion_q.get()
                  ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
          ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 430, in _recv_bytes
    buf = self._recv(4)
          ^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 395, in _recv
    chunk = read(handle, remaining)
            ^^^^^^^^^^^^^^^^^^^^^^^
K

In [ ]:
training_q.empty()

True

In [ ]:
from accelerate.data_loader import DataLoaderShard

In [ ]:
from torch.utils.data import IterableDataset, _DatasetKind
temp_iterator = iter(trainer.get_train_dataloader())

In [ ]:
next(temp_iterator)

StopIteration: 

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/simpissa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /home/simpissa/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/simpissa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /home/simpissa/nltk_data...
[nltk_data]   Package words is already up-to-date!


Observation: ["\n[GAME] You are Playing Wordle.\nA secret 5-letter word has been chosen. You have 6 attempts to guess it.\nFor each guess, wrap your word in square brackets (e.g., '[apple]').\nFeedback for each letter will be given as follows:\n  - G (green): correct letter in the correct position\n  - Y (yellow): letter exists in the word but in the wrong position\n  - X (wrong): letter is not in the word\nEnter your guess to begin.\n", "\n[GAME] You are Playing Wordle.\nA secret 5-letter word has been chosen. You have 6 attempts to guess it.\nFor each guess, wrap your word in square brackets (e.g., '[apple]').\nFeedback for each letter will be given as follows:\n  - G (green): correct letter in the correct position\n  - Y (yellow): letter exists in the word but in the wrong position\n  - X (wrong): letter is not in the word\nEnter your guess to begin.\n"]
